### 多路召回
多路召回指的是采用不同的策略、特征或者简单模型，分别召回一部分候选集，然后把候选集混合在一起供后续排序模型使用。<br>
可以使用多种不同的策略来获取用户排序的候选商品集合，而具体使用的策略是和业务强相关的，针对不同的任务就会有对于该业务真实场景下需要考虑的召回规则。<br>
例如，新闻推荐的召回规则可以是“热门新闻”、“作者召回”、“关键词召回”、“主题召回”、“协同过滤召回”。

## 导包

In [18]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import pickle

In [2]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/news-recommendation-dataset/articles.csv
/kaggle/input/news-recommendation-dataset/testA_click_log.csv
/kaggle/input/news-recommendation-dataset/train_click_log.csv
/kaggle/input/news-recommendation-dataset/articles_emb.csv
/kaggle/input/news-recommendation-dataset/sample_submit.csv


In [23]:
data_path='/kaggle/input/news-recommendation-dataset/'
save_path='/kaggle/output/'
metric_recall=False

## 读取数据
1. debug模式：基于数据搭建baseline并且跑通，可以在debug模式下从海量的数据集中随机抽取一部分样本（**train_click_log_sample**）来对baseline进行调试；
2. 线下验证模式：基于已有的训练集数据，来选择好合适的模型和一些超参数，因此这一块只需要加载整个训练集（**train_click_log**），然后将训练集分为训练集和验证集。（训练集是模型的训练数据，验证集部分帮助我们调整模型的参数和其他的一些超参数）
3. 线上模式：用debug模式搭建好一个推荐系统比赛的baseline，用线下验证模式选择好了模型和一些超参数，这一部分就是对于给定的测试集进行预测，提交到线上，所以这块使用的训练数据集是全量的数据集（**train_click_log+test_click_log**）

In [14]:
# debug模式：从训练集中划分一些数据来调试代码
def get_all_click_sample(data_path, sample_nums=10000):
    """
        从训练数据集中随机抽取一部分数据用于调试
        data_path：原数据的存储路径
        sample_nums：采样数目
    """
    all_click=pd.read_csv(data_path+'train_click_log.csv')
    all_user_ids=all_click.user_id.unique()
    
    # 随机抽取sample_nums个数据
    sample_user_ids=np.random.choice(all_user_ids,size=sample_nums,replace=False)
    # 获取sample_user_ids对应的在all_click中的数据
    all_click=all_click[all_click['user_id'].isin(sample_user_ids)]
    
    all_click=all_click.drop_duplicates((['user_id','click_article_id','click_timestamp']))
    return all_click

# 读取点击数据
def get_all_click_df(data_path,offline=True):
    """
        从给定的路径中读取点击数据，并根据offline参数决定是仅读取训练数据还是同时读取训练和测试数据
        data_path：原数据的存储路径
        offline：表示是否处于离线模式。在离线模式下，只处理训练数据，否则，同时处理训练和测试数据
    """
    if offline:
        all_click=pd.read_csv(data_path+'train_click_log.csv')
    else:
        trn_click=pd.read_csv(data_path+'train_click_log.csv')
        tst_click=pd.read_csv(data_path+'testA_click_log.csv')
        
        # 包含测试集和训练集
        all_click=pd.concat([trn_click,tst_click])
    
    # 去除重复的点击记录，保留唯一的(user_id, click_article_id, click_timestamp)组合
    all_click=all_click.drop_duplicates((['user_id','click_article_id','click_timestamp']))
    return all_click

In [6]:
# 读取文章的基本属性
def get_item_info_df(data_path):
    item_info_df=pd.read_csv(data_path+'articles.csv')
    
    # 为了与训练集中的click_article_id进行拼接，修改article_id为click_article_id
    item_info_df=item_info_df.rename(columns={'article_id':'click_article_id'})
    
    return item_info_df

In [7]:
# 读取文章的embedding属性
def get_item_emb_dict(data_path):
    item_emb_df=pd.read_csv(data_path+'articles_emb.csv')
    
    item_emb_cols=[x for x in item_emb_df.columns if 'emb' in x]
    item_emb_np=np.ascontiguousarray(item_emb_df[item_emb_cols])
    # 进行归一化
    item_emb_np=item_emb_np/np.linalg.norm(item_emb_np,axis=1,keepdims=True)
    
    item_emb_dict=dict(zip(item_emb_df['article_id'],item_emb_np))
    pickle.dump(item_emb_dict,open(save_path+'item_content_emb.kpl','wb'))
    
    return item_emb_dict

In [8]:
max_min_scaler=lambda x: (x-np.min(x))/(np.max(x)-np.min(x))

In [10]:
# 采样数据
all_click_df=get_all_click_sample(data_path)
all_click_df.head()

,user_id,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
14,199994,235230,1507053409054,4,1,17,1,8,2
15,199994,271551,1507055176032,4,1,17,1,8,2
16,199994,236444,1507292778083,4,1,17,1,8,2
17,199994,236951,1507295091338,4,1,17,1,8,2
18,199994,205973,1507300936624,4,1,17,1,8,2


In [15]:
# 全量训练集
all_click_df=get_all_click_df(data_path,offline=False)
all_click_df.head()

,user_id,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,199999,160417,1507029570190,4,1,17,1,13,1
1,199999,5408,1507029571478,4,1,17,1,13,1
2,199999,50823,1507029601478,4,1,17,1,13,1
3,199998,157770,1507029532200,4,1,17,1,25,5
4,199998,96613,1507029671831,4,1,17,1,25,5


In [16]:
item_info_df=get_item_info_df(data_path)
item_info_df.head()

,click_article_id,category_id,created_at_ts,words_count
0,0,0,1513144419000,168
1,1,1,1405341936000,189
2,2,1,1408667706000,250
3,3,1,1408468313000,230
4,4,1,1407071171000,162


In [24]:
item_emb_dict=get_item_emb_dict(data_path)
item_emb_dict.head()

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/output/item_content_emb.kpl'

## 工具函数

### 获取用户-文章-时间函数

In [27]:
# 根据点击时间获取用户的点击文章序列：{user1:[(item1,time1),(item2,time2)]}
def get_user_item_time(click_df):
    """
        创建一个字典，其中的键是商品的id，值是商品被不同用户点击的时间序列
        其中，这个时间序列是一个列表，列表中的每个元素是一个元组，元组包含用户id和点击时间
        click_df：包含用户的点击数据，有click_article_id, click_timestamp
    """
    click_df=click_df.sort_values('click_timestamp')
    def make_item_time_pair(df):
        """
            辅助函数，用于将点击数据中的click_article_id和click_timestamp列打包成一个元组列表
            使用zip函数将两列的值组合成一个元组列表
        """
        return list(zip(df['click_article_id'],df['click_timestamp']))
    
    
    user_item_time_df=click_df.groupby('user_id')['click_article_id','click_timestamp'].apply(lambda x: make_item_time_pair(x)).reset_index().rename(columns={0:'item_time_list'})
    
    user_item_time_dict=dict(zip(user_item_time_df['click_article_id'],user_item_time_df['user_time_list']))
    return user_item_time_df

### 获取文章-用户-时间函数

In [ ]:
# 根据时间获取商品被点击的用户序列: {item1:[(user1,time1),(user2,tim2),...]}
def get_item_user_time_dict(click_df):
    def make_user_time_pair(df):
        return list(zip(df['user_id'],df['click_timestamp']))
    
    click_df=click_df.sort_values('click_timestamp')
    item_user_time_df=click_df.groupby('click_article_id')['user_id','click_timestamp'].apply(lambda x: make_user_time_pair(x)).reset_index().rename(columns={0:'user_time_list'})
    
    item_user_time_dict=dict(zip(item_user_time_df['click_article_id'],item_user_time_df['user_time_list']))
    return item_user_time_dict

### 获取历史和最后一次点击
在评估召回结果，特征工程和制作标签转化成监督学习测试集时可以使用

In [ ]:
# 获取当前数据的历史点击和最后一次点击
def get_hist_and_last_click(all_click):
    """
        返回两个df：一个是用户的历史点击记录（不包括最后一次点击）
                  一个是用户的最后一次点击记录
    """
    all_click=all_click.sort_values(by=['user_id','click_timestamp'])
    click_last_df=all_click.groupby('user_id').tail(1)
    
    def hist_func(user_df):
        """
            获取每个用户的历史点击记录（不包括最后一次点击）
            如果用户只有一次点击记录，则直接返回这个记录
            如果用户有多次点击记录，返回除了最后一次点击之外的所有记录
        """
        if len(user_df)==1:
            return user_df
        else:
            return user_df[:-1]
    
    # 获取历史点击
    click_hist_df=all_click.groupby('user_id').apply(hist_func).reset_index(drop=True)
    
    # click_hist_df：每个用户的历史点击记录
    # click_last_df：包含每个用户的最后一次点击记录
    return click_hist_df, click_last_df

### 获取文章属性特征

In [28]:
# 获取文章id对应的基本属性，保存成字典的形式，方便后面召回阶段，冷启动阶段使用
def get_item_info_dict(item_info_df):
    """
        从一个包含文章（或者商品）信息的df中提取每篇文章的特定属性，并将其保存为字典形式
        提取每篇文章的类别id、词数和创建时间，并且将他们保存在字典中
    """
    max_min_scaler=lambda x: (x-np.min(x))/(np.max(x)-np.min(x))
    # 归一化时间
    item_info_df['created_at_ts']=item_info_df['created_at_ts'].apply(max_min_scaler)
    
    # 将click_article_id作为键，category_id作为值
    item_type_dict=dict(zip(item_info_df['click_article_id'],item_info_df['category_id']))
    # 将click_article_id作为键，words_count作为值
    item_words_dict=dict(zip(item_info_df['click_article_id'],item_info_df['words_count']))
    # 将click_article_id作为键，归一化后的created_at_ts作为值，创建一个字典
    item_created_time_dict=dict(zip(item_info_df['click_article_id'],item_info_df['created_at_ts']))
    
    # item_type_dict：文章ID到类别ID的映射
    # item_words_dict：文章ID到词数的映射
    # item_created_time_dict：文章ID到归一化创建时间的映射
    return item_type_dict, item_words_dict, item_created_time_dict

### 获取用户历史点击的文章信息

In [29]:
def get_user_hist_item_info_dict(all_click):
    
    # 获取user_id对应的用户历史点击文章类型的集合字典
    user_hist_item_typs=all_click.groupby('user_id')['category_id'].agg(set).reset_index()
    user_hist_item_typs_dict=dict(zip(user_hist_item_typs['user_id'],user_hist_item_typs['category_id']))
    
    # 获取user_id对应的用户点击文章的集合
    user_hist_item_ids_dict=all_click.groupby('user_id')['click_article_id'].agg(set).reset_index()
    user_hist_item_ids_dict=dict(zip(user_hist_item_ids_dict['user_id'],user_hist_item_ids_dict['click_article_id']))
    
    # 获取user_id对应的用户历史点击的文章的平均字数字典
    user_hist_item_words=all_click.groupby('user_id')['words_count'].agg('mean').reset_index()
    user_hist_item_words_dict=dict(zip(user_hist_item_words['user_id'],user_hist_item_words['words_count']))
    
    # 获取user_id对应的用户最后一次点击的文章的创建时间
    all_click_ = all_click.sort_values('click_timestamp')
    user_last_item_created_time = all_click_.groupby('user_id')['created_at_ts'].apply(lambda x: x.iloc[-1]).reset_index()
    
    max_min_scaler = lambda x : (x-np.min(x))/(np.max(x)-np.min(x))
    user_last_item_created_time['created_at_ts'] = user_last_item_created_time[['created_at_ts']].apply(max_min_scaler)
    
    user_last_item_created_time_dict = dict(zip(user_last_item_created_time['user_id'], \
                                                user_last_item_created_time['created_at_ts']))
    
    return user_hist_item_typs_dict, user_hist_item_ids_dict, user_hist_item_words_dict, user_last_item_created_time_dict

### 获取点击次数最多的topk个文章

In [30]:
def get_item_topk_click(click_df,k):
    topk_click=click_df['click_article_id'].value_counts().index[:k]
    return topk_click

## 定义多路召回字典

In [31]:
# 获取文章的属性信息，保存成字典的形式方便查询
item_type_dict, item_words_dict, item_created_time_dict=get_item_info_dict(item_info_df)

/tmp/ipykernel_30/918158262.py:3: RuntimeWarning: invalid value encountered in scalar divide
  max_min_scaler=lambda x: (x-np.min(x))/(np.max(x)-np.min(x))


In [32]:
# 定义一个多路召回的字典，将各路召回的结果都保存在这个字典中
user_multi_recall_dict={'itemcf_sim_itemcf_recall':{},
                        'embedding_sim_item_recall':{},
                        'youtubednn_recall':{},
                        'youtubednn_usercf_recall':{},
                        'cold_start_recall':{}}

In [ ]:
# 如果需要做召回评估，则提取最后一次点击作为召回评估
# 如果不需要做召回评估，则使用全量的训练集进行召回（线下验证模型）
trn_hist_click_df, trn_last_click_df = get_hist_and_last_click(all_click_df)

## 召回效果评估函数

In [33]:
# 依次评估召回的前10, 20, 30, 40, 50个文章中的击中率
def metrics_recall(user_recall_items_dict, trn_last_click_df, topk=5):
    last_click_item_dict = dict(zip(trn_last_click_df['user_id'], trn_last_click_df['click_article_id']))
    user_num = len(user_recall_items_dict)
    
    for k in range(10, topk+1, 10):
        hit_num = 0
        for user, item_list in user_recall_items_dict.items():
            # 获取前k个召回的结果
            tmp_recall_items = [x[0] for x in user_recall_items_dict[user][:k]]
            if last_click_item_dict[user] in set(tmp_recall_items):
                hit_num += 1
        
        hit_rate = round(hit_num * 1.0 / user_num, 5)
        print(' topk: ', k, ' : ', 'hit_num: ', hit_num, 'hit_rate: ', hit_rate, 'user_num : ', user_num)

## 计算相似性矩阵
通过协同过滤和向量检索得到相似性矩阵，相似性矩阵主要分为user2user和item2item

In [ ]:
def itemcf_sim(df,item_created_time_dict):
    """
        文章与文章之间的相似性矩阵计算-->基于物品的协同过滤
        df:数据表
        item_created_time_dict:文章创建时间的字典
        return: 文章与文章的相似性矩阵
    """
    user_item_time_dict=get_user_item_time(df)
    
    # 计算物品相似度
    i2i_sim={}
    item_cnt=defaultdict(int)
    for user,item_time_list in tqdm(user_item_time_dict.items):
        # 在基于商品的协同过滤优化的时候可以考虑时间因素
        for loc1, (i,i_click_time) in enumerate(item_time_list):
            item_cnt[i]+=1
            i2i_sim.setdefault(i,{})
            for loc2,(j,j_click_time) in enumerate(item_time_list):
                if (i==j):
                    continue
                
                # 考虑文章的正向顺序点击和反向顺序点击
                loc_alpha=1.0 if loc2>loc1 else 0.7